In [1]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt

Without air resistance, the x and y coordinates of the ball follow
$$
\begin{align}
y(t) &= v_i \sin(\theta) t - \frac{1}{2} g t^2 \\
x(t) &= v_i \cos(\theta) t
\end{align}
$$

Thus, we can express y in terms of x as such:
$$
\begin{align}
y(t) &= x \tan(\theta) - \frac{g}{2 v_i^2 \cos^2(\theta)} x^2 \\
\end{align}
$$

and the range of the projectile as
$$
d=\frac{2 v_i^2 \sin(\theta) \cos(\theta)}{g}=\frac{v_i ^2 \sin(2 \theta)}{g}
$$

The code below plots this ideal range against some parameters

In [ ]:
g = 9.81

# range against initial velocity

theta = np.pi / 4
v_values = np.linspace(0, 10, 50)
d_values = np.power(v_values, 2) * np.sin(2 * theta) / g

plt.plot(v_values, d_values)
plt.show()

# range against angle

v = 1
theta_values = np.linspace(0, np.pi / 2, 50)
d_values = np.power(v, 2) * np.sin(2 * theta_values) / g

plt.plot(theta_values, d_values)
plt.show()

With air resitance, the equation of motion (vector form) is:
$$
m\vec{a} = m\vec{g} - k|\vec{v_{rel}}|^2 \hat{v_{rel}}
$$
Where $\vec{v_{rel}}=\vec{v}-\vec{v_{wind}}$ is the relative velocity of the ball to the wind

Splitting into components and dividing by m:
$$
\begin{align}
\ddot{x} &= -\frac{k}{m} \left(\left(\dot{x}-v_{wind}\right)^2+\dot{y}^2\right) \frac{\dot{x}-v_{wind}}{\sqrt{\left(\dot{x}-v_{wind}\right)^2+\dot{y}^2}} \\
\ddot{y} &= -mg-\frac{k}{m} \left(\left(\dot{x}-v_{wind}\right)^2+\dot{y}^2\right) \frac{\dot{y}}{\sqrt{\left(\dot{x}-v_{wind}\right)^2+\dot{y}^2}}
\end{align}
$$

Note that $m$ and $k$ only appear in the form $\frac{k}{m}$, so we can redefine $k$ to be $\frac{k}{m}$:
$$
\begin{align}
\ddot{x} &= -k \left(\left(\dot{x}-v_{wind}\right)^2+\dot{y}^2\right) \frac{\dot{x}-v_{wind}}{\sqrt{\left(\dot{x}-v_{wind}\right)^2+\dot{y}^2}} \\
\ddot{y} &= -mg-k \left(\left(\dot{x}-v_{wind}\right)^2+\dot{y}^2\right) \frac{\dot{y}}{\sqrt{\left(\dot{x}-v_{wind}\right)^2+\dot{y}^2}}
\end{align}
$$

The code below solves this ODE and plots its solution

In [2]:
def projectile_launch(t, vec, k, g, v_wind):
    x, xp, y, yp = vec
    return [
        xp,
        - k * (xp - v_wind) * np.sqrt(np.power(xp - v_wind, 2) + np.power(yp, 2)),
        yp,
        - g - k * yp * np.sqrt(np.power(xp - v_wind, 2) + np.power(yp, 2))
    ]

In [ ]:
k = 0.1
g = 9.81
v_wind = -10

v = 100
theta = np.pi / 4
initial_vec = [0, v * np.cos(theta), 0, v * np.sin(theta)]

# solution is stored as an array of x and y values
sol_list = [[], []]
dt = 0.001

# integrating the ode
sol = integrate.ode(projectile_launch)
sol.set_initial_value(initial_vec, 0).set_f_params(k, g, v_wind)

while sol.successful() and sol.y[2] >= 0:
    sol_list[0].append(sol.y[0])
    sol_list[1].append(sol.y[2])

    sol.integrate(sol.t + dt)

In [ ]:
plt.plot(sol_list[0], sol_list[1])
plt.show()

Generating data:

In [54]:
k = 0.1
g = 9.81
v_wind = 4 + 0.3 * (np.random.random() - 0.5)

v = 4 + 0.1 * (np.random.random() - 0.5)
theta = 0.25 * np.pi + 0.05 * (np.random.random() - 0.5)
initial_vec = [0, v * np.cos(theta), 0, v * np.sin(theta)]

# solution is stored as an array of x and y values
data = []
dt = 0.001
num_steps = 1 / 60 / dt
n = 0

# integrating the ode
sol = integrate.ode(projectile_launch)
sol.set_initial_value(initial_vec, 0).set_f_params(k, g, v_wind)

while sol.successful() and sol.y[2] >= 0:
    n += 1
    if n >= num_steps:
        data.append([sol.t, sol.y[0] + 0.01 * (np.random.random() - 0.5), sol.y[2] + 0.01 * (np.random.random() - 0.5)])
        n = 0

    sol.integrate(sol.t + dt)

with open("data/projectile launcher/set14/data3.csv", "w") as f:
    np.savetxt(f, data, fmt="%.3f", delimiter=",")